In [1]:
from gp_constraint_min import BayeOptConstraint
from objective_function import objective_function, objective_function_products,print_result
from skopt import gp_minimize
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import ConstantKernel, Matern,RBF


import numpy as np
import random
from BO_min import BayeOpt
from objective_function import objective_function_penalty

def obj(x):
    res = objective_function_penalty(x,penalty = 10000000, target_service_level = 0.8, days = 300, plotdata = False)
    return res


def f_product(x):
    res = objective_function_products(x, days = 300, plotdata = False, return_SL = False)
    return res#logit(res)
def f_cost(x):
    res = objective_function(x, days = 300, plotdata = False, return_SL = False)
    return res
def initial_points(bounds, nums):
    random.seed(0)
    X_init = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(nums, bounds.shape[0]))
    return X_init
bounds = np.array([[100,400] for _ in range(12)])
X_init = initial_points(bounds, 20)

In [31]:
from objective_function import get_sim_result

In this notebook, I compared two strategies

### New EI function

In [4]:
m52_1 = ConstantKernel(1) * RBF(np.array([100]*12))
gpr1 = GaussianProcessRegressor(kernel=m52_1, alpha=10, noise = "gaussian")

m52_2 = ConstantKernel(1) *  RBF(np.array([100]*12))
gpr2  = GaussianProcessRegressor(kernel=m52_2, alpha=10, noise = "gaussian")


In [16]:
min1 = []
min1_x = []
min2 = []
min2_x = [] 
for i in range(5):
    # first
    X_init = initial_points(bounds, 10)
    m52_1 = ConstantKernel(1) * RBF(np.array([100]*12))
    gpr1 = GaussianProcessRegressor(kernel=m52_1, alpha=10, noise = "gaussian")

    m52_2 = ConstantKernel(1) *  RBF(np.array([100]*12))
    gpr2  = GaussianProcessRegressor(kernel=m52_2, alpha=10, noise = "gaussian")
    
    A = BayeOptConstraint(obj_func = f_cost,
             constraint_func = f_product,
             constraint_value = 0,
             bounds = bounds, 
             gpr_obj = gpr1,
             gpr_constraint = gpr2,
             init_x = X_init, 
             niter = 80,
             n_restarts = 25)
    f_min1,f_x = A.bay_opt()
    min1.append(f_min1)
    min1_x.append(f_x)
    # second
    m52_1 = ConstantKernel(1) * RBF(np.array([100]*12))
    gpr1 = GaussianProcessRegressor(kernel=m52_1, alpha=10, noise = "gaussian")
    B = BayeOpt(obj_func = obj,
             bounds = bounds, 
             gpr_obj = gpr1,
             init_x = X_init, 
             niter = 80,
             n_restarts = 25)
    f_min2,f_x2 = B.bay_opt()
    min2.append(f_min2)
    min2_x.append(f_x2)
    

    

(10, 1) [[12500545.70392506]
 [14091242.55206754]
 [13591770.63652584]
 [13420851.4355999 ]
 [12750376.00023696]
 [13464999.30029019]
 [13451743.49926995]
 [13381368.27143199]
 [13954547.07704857]
 [13635250.76445827]]
(10, 1) [[-8380.29410068]
 [-1963.9371881 ]
 [-5204.66730081]
 [-8411.06495609]
 [   58.54940661]
 [ -974.26158632]
 [-8663.57656628]
 [-6844.81218038]
 [-8492.14627474]
 [-8444.83926506]]
(10, 1) [[8.61652523e+10]
 [1.71654685e+10]
 [6.79296261e+10]
 [1.04439923e+11]
 [1.27331782e+07]
 [5.13906318e+09]
 [8.49312174e+10]
 [7.28491690e+10]
 [8.02540626e+10]
 [1.23372078e+11]]
(10, 1) [[12808251.06173686]
 [13611904.74482807]
 [12533698.72776785]
 [13631199.77812617]
 [12587277.2875379 ]
 [13449255.42630913]
 [13689318.42751892]
 [12767390.84270187]
 [13736428.8112359 ]
 [13449401.15700639]]
(10, 1) [[-6463.13030285]
 [-8573.96301285]
 [-8578.12295476]
 [   60.6       ]
 [-8558.79988465]
 [-8411.21560849]
 [-8342.37560389]
 [-3973.63550877]
 [  -87.26508832]
 [-8358.480244

In [27]:
min1

[array([12750376.00023696]),
 array([12070304.713602]),
 array([12507845.91620451]),
 array([12938096.00639022]),
 array([12894271.11586309])]

In [28]:
min2

[12556195.27181442,
 12735707.63953354,
 12527132.605293691,
 13145282.956668839,
 12267280.353714282]

In [29]:
print("New EI: mean", np.mean(min1), "std", np.std(min1))

New EI: mean 12632178.750459354 std 318562.2238440728


In [30]:
print("Penalty: mean",np.mean(min2), "std",np.std(min2))

Penalty: mean 12646319.765404955 std 290844.74806075386


In [32]:
def compute_servicelevel(x):
    WarehouseA = get_sim_result(x, days = 300, plotdata = False)
    
    minSL = 1
    for i in range(1,7):
        servicelevel = 1 - WarehouseA.totalLateSales["Product" + str(i)]/ WarehouseA.totalDemand["Product" + str(i)]
        minSL = min(minSL,servicelevel )
            
    return minSL

In [52]:
minSL1 = [compute_servicelevel(x) for x in min1_x]

In [53]:
minSL2 = [compute_servicelevel(x) for x in min2_x]

In [54]:
minSL1

[0.8945937220385741,
 0.866083685388236,
 0.920359051740727,
 0.8756778871466252,
 0.8287178141708744]

In [55]:
minSL2

[0.9003160870624451,
 0.8610808556111279,
 0.9060659068005706,
 0.9801563439567047,
 0.9301505028714064]